In [1]:
# Objectives 

    # Quantify uncertainty 
    # Model based on uncertainty 
    # Provide answers in the form of distributions (not point estimates)

In [2]:
from scipy import stats
import arviz as az # exploratory bayesian analysis, interpret + visualize posterior distributions
import numpy as np 
import matplotlib.pyplot as plt 
import pymc3 as pm # probabilistic programming 
import seaborn as sns
import pandas as pd 
from theano import shared 
from sklearn import preprocessing 

data = pd.read_csv('../input/renfe.csv')
data.drop ('Unnamed: 0', 
          axis = 1, 
          inplace = True)

data = data.sample(frac = 0.01,
                  random_state = 99)

ModuleNotFoundError: No module named 'arviz'

In [3]:
data.head(3)

NameError: name 'data' is not defined

In [4]:
# Check completeness 
data.isnull().sum()/len(data)

NameError: name 'data' is not defined

In [5]:
# 3 categories contain missing variables 
    # 12% price

In [6]:
# Fill missing class with most common class
# Fill missing fare type with most common fare type 
# Fill missing price with mean of their respective fare types 

data['train_class'] = data['train_class'].fillna(data['train_class'].mode().iloc[0])

data['fare'] = data['fare'].fillna(data['fare'].mode().iloc[0])

data['price'] = data.groupby('fare').transform(lambda x:x.fillna(x.mean()))

NameError: name 'data' is not defined

In [7]:
for i in ['start_date', 'end_date']:
    data[i] = pd.to_datetime(data[i])
    
data.shape

NameError: name 'pd' is not defined

In [8]:
# Gaussian inference 
    # Price across dataset
        # not categorized according to fare type 
    
# KDE - show density
az.plot_kde(data['price'].values, rug = True)
plt.yticks([0], alpha = 0);

NameError: name 'az' is not defined

In [9]:
# kde plot shows normal / Gaussian like distribution 

# Set PRIORS for std and mean (unknown parameters) reflecting ignorance 

    # Mu = upper and lower boundaries in normal distribution 

    # Theta = can only be positive std, halfnormal distribution 
    
# Price likelihood function:
    
    # Likelihood function = new evidence or information contained in observed data

    # y = observed variable representing data with normal distribution with above params 
    
    # y specifies the likelihood
    
    # y Informs PyMC3 to condition for the unknown on the known data.
    
    # Draw 1000 POSTERIOR samples using NUTS sampling 
    
        # NUTS = No U-Turn Sampler

In [10]:
# Model using PyMC3
    # Set priors
        # Compute posterior 
            # Check maximum posterior estimates are close to true params 

with pm.Model() as model_g:
    mu = pm.Uniform('mu', lower = 0, upper = 300) # prior estimates
    theta = pm.HalfNormal('theta', sd = 10)
    y = pm.Normal('y', mu = mu, sd = theta, observed = data['price'].values) # condition unknown mu and theta on known prices
    trace_g = pm.sample(1000, tune = 1000)

NameError: name 'pm' is not defined

In [11]:
az.plot_trace(trace_g);

NameError: name 'az' is not defined

In [12]:
# 1 plot row for each param
    # Posterior is bi-dimensional / bi-variate data
    # Shows the marginal distributions of each param
        # X or Y of theta / mu 

# Left = KDE 
    # parameter value on x-axis
    # parameter probability on y-axis 

# Right = Trace plot 
    # individual sampled values at each step of sampling 
    # Visualize plausible values from the posterior 
    
    
# Sampling chains for each param (KDE) seems well-converged 
    # chains appear stationary (no odd patterns / drifts)
    
# Maximum posterior estimate of each param very close to true params 
    # Peaks in KDE 

In [13]:
# Joint distribution of params 
    # Check correlation & collinearity (correlation between independent variables)

az.plot_joint(trace_g, kind = 'kde', fill_last = False);

NameError: name 'az' is not defined

In [14]:
# No correlation between parameters 
    # = No collinearity in model 

In [15]:
# Posterior dist of each param 

az.summary(trace_g)

NameError: name 'az' is not defined

In [16]:
# Posterior interpretation - Bayesian inference 

# Visualize mean and HPD of a dist
    # HPD = Highest Posterior Density [interval]
    
    # HPD = shortest interval on a posterior density for a given confidence level
        # =/= confidence intervals
    
# Use mu, theta, HPD 94% (HPD 97 and 3)

az.plot_posterior(trace_g);

NameError: name 'az' is not defined

In [17]:
# Bayesian stats gives entire dist of values 

# ArviZ uses default 94% HPD

# Interpretation:
    # 94% probability of : 
        # 63.8 < mu of price < 64.4
        # 24.5 < theta of price < 24.9

In [18]:
# Verify convergence of sampling chains 
    # Using Gelman Rubin convergence diagnostic 
        # Ideal = values close to 1.0 mean convergence 
        
# Gelman-Rubin compares within-chain var to between-chain var

pm.gelman_rubin(trace_g)

NameError: name 'pm' is not defined

In [19]:
# Visualize Bayesian Fraction of Missing Information (BFMI) & Gelman-Rubin 

    # Ideal result = close convergence of chains 
        # If poor convergence, model is misspecified 
        
bfmi = pm.bfmi(trace_g)

max_gr = max(np.max(gr_stats) for gr_stats in pm.gelman_rubin(trace_g).values())

(pm.energyplot(trace_g, legend = False, figsize = (6, 4)).set_title("BFMI = {}\nGelman-Rubin = {}".format(bfmi, max_gr)));

NameError: name 'pm' is not defined

In [20]:
# Belief model on mean price and theta converged well and agrees with original data 
# statistics close to 1 

In [21]:
# Posterior predictive checks (PPC)
    # Validate model 

# Generate data from model using params from computed posterior 
    # Use simulation results to derive predictions 
    
    
# Create function to randomly draw 1,000 samples of params from trace. 
    # for each sample draw 25798 random numbers from a normal dist
        # normal dist is specified by mu and theta values in each sample
        
ppc = pm.sample_posterior_predictive(trace_g,
                                    samples = 1000,
                                    model = model_g)

NameError: name 'pm' is not defined

In [22]:
# Check ppc contains 1000 generated datasets
    # datasets containing 25798 samples each
        # each sample using different param settings from posterior estimate
        
np.asarray(ppc['y']).shape

NameError: name 'np' is not defined

In [23]:
# Plot ppc

_, ax = plt.subplots(figsize =(10, 5))

ax.hist([y.mean() for y in ppc['y']], 
        bins = 19, alpha = 0.5)

ax.axvline(data.price.mean())

ax.set(title = 'Posterior predictive of mean',
      xlabel = 'mean(x)',
      ylabel = 'Freq');

NameError: name 'plt' is not defined

In [24]:
# Inferred mean very close to actual price mean

In [25]:
# Group comparison  
    # Quantify price differences between fare types 
        # using mean of each fare type 
        
# Obtain posterior distribution of differences in means between fare types
    # Posterior dist = summarizes knowledge about uncertain quantities 
        # = combination of likelihood function and prior distribution
        
# Show differences between mean of price varies according to fare types 

sns.boxplot(x = 'fare', y = 'price', data = data);
    

NameError: name 'sns' is not defined

In [26]:
data.fare.unique()

NameError: name 'data' is not defined

In [27]:
# Create 3 variables 
    # price / categorical dummy encoding fare types / group variable containing fare types 

price = data['price'].values # for convenience 
    
fare_dummy = pd.Categorical(data['fare'],
                               categories =['Flexible',
                                           'Promo',
                                           'Promo +',
                                           'Adulto ida',
                                           'Mesa',
                                           'Individual-Flexible']).codes
    
groups = len(np.unique(fare_dummy))


NameError: name 'data' is not defined

In [28]:
# Estimate model params per fare type / group 

# mu and theta are vectors (instead of scalar variables as in model_g)

# Pass shape argument for setting priors (since params = vectors)

# Index mu and theta using fare_dummy for generating y (likelihood function)
    # where data = price 

with pm.Model() as group_comparison:
    mu = pm.Normal('mu', 
                   sd = 10, shape = groups)
    theta = pm.HalfNormal('theta', 
                          sd = 10, shape = groups)
    y = pm.Normal('y', 
                  mu = mu[fare_dummy], sd = theta[fare_dummy], observed = price)
    trace_groups = pm.sample(5000, tune = 5000)

NameError: name 'pm' is not defined

In [29]:
# Plot PPC model 

az.plot_trace(trace_groups);

NameError: name 'az' is not defined

In [30]:
# kde plot shows likelihood of params (y) at each param value (x)
    # For all fare types / group

# well converged, no drifts or oddities 

In [31]:
# Show fare group table
    # View differences

fares_table = az.from_pymc3(trace = trace_groups)

fares_gaussian = az.summary(fares_table)

fares_gaussian

NameError: name 'az' is not defined

In [32]:
# Large difference between mean & sd for every group (fare type)

In [33]:
# Plot differences in mean and HPD for every group
    # 5 col, 2 rows 
        # 5 groups, bi-variate / bi-dimensional data 
        
# Use cohen's d to indicate differences between two means 
    # = difference between mu divided by pooled theta 

dist = stats.norm()

_, ax = plt.subplots(5, 2, figsize = (20, 12), 
                     constrained_layout = True)

comparisons = [(i, j) for i in range(6) for j in range(i+1, 6)]

pos = [(k, l) for k in range(5) for l in (0, 1)]

for (i, j), (k, l) in zip (comparisons, pos):
    means_diff = trace_groups['mu'][:, i] - trace_groups['mu'][:, j]
    d_cohen = (means_diff / np.sqrt((trace_groups['theta'][:, i]**2 + trace_groups['theta'][:, j]**2) / 2)).mean()
    ps = dist.cdf(d_cohen/(2**0.5))
    az.plot_posterior(means_diff, ref_val = 0, ax = ax[k, l])
    ax[k, l].set_title(f'$\mu_{i}-\mu_{j}$')
    ax[k, l].plot(0, label = f"Cohen's d = {d_cohen:.2f}\nProb sup = {ps:.2f}",
                 alpha = 0)
    ax[k, l].legend();

NameError: name 'plt' is not defined

In [34]:
# All plots show the comparisons do not include reference value of 0 
    # given 94% HPD 
    
# Rule out null-hypothesis of mu1 = mu2 

# Evidence to conclude range difference (6.1 < mu < 63.5)
    # large enough to justify purchases are based on different fare types 

In [35]:
# Bayesian Hierarchal Linear Regression 